In [ ]:
import rp
import source.clip as clip
import torch
from tqdm import tqdm

In [ ]:
device=rp.select_torch_device(prefer_used=True)

In [ ]:
# image='https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG'
image='https://i.guim.co.uk/img/media/380500f7fd9321a6cfd1bd1a3b3f104ad9797bb7/0_0_2000_1199/master/2000.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f0d0ecb5a0c33aa91be270b08619e8d7'

# image='https://www.cs.stanford.edu/~roozbeh/pascal-context/2008_001997.jpg'
# image='https://www.cs.stanford.edu/~roozbeh/pascal-context/2008_002123.jpg'

image='https://deadline.com/wp-content/uploads/2016/06/finding-dory.jpg?w=620&h=383&crop=1'

image='https://people.com/thmb/3XqXKvbM1X9DzblQMy7djsaIvck=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc():focal(614x89:616x91)/danny-devito-arnold-schwarzenegger-022023-93d53e2180544ebe9f5607870aadbf3d.jpg'

image='https://www.rollingstone.com/wp-content/uploads/2022/11/Kim-Taylor.jpg?w=831&h=554&crop=1' #petras/swift

size=224

image=rp.load_image(image)
image=rp.resize_image_to_hold(image,height=size,width=size,allow_shrink=True)
image=rp.as_float_image(image)
image=rp.as_rgb_image(image)
rp.display_image(image)

In [ ]:
def shotgun_crop(image,min_scale=.1,max_scale=.8):
    height,width=rp.get_image_dimensions(image)
    scale=rp.random_float(min_scale,max_scale)
    height=int(height*scale)
    width=int(width*scale)
    return rp.crop_image_at_random_position(image,height,width)

In [ ]:
cropped_images=[shotgun_crop(image) for _ in range(2000)]

In [ ]:
with torch.no_grad():
    scores = []
    for x in tqdm(cropped_images):
        clip_logits = clip.get_clip_logits(
            rp.as_torch_image(x).to(device),
            [
                # 'harry potter'
                # 'hermoine'
                # 'nemo'
                # 'dory'
                # 'chair'
                # 'helmet'
                # 'blue jeans'
                # 'monitor'
                # 'danny devito'
                # 'Arnold Schwarzenegger'
                # 'kim petras'
                'taylor swift'
            ]
        )[0]
        score = float(clip_logits)
        scores.append(score)

In [ ]:
rp.histogram_via_bokeh(scores, bins=100)

In [ ]:
d_scores,_,scored_images=rp.sync_sort(scores,list(range(len(scores))),cropped_images)

d_height, d_width=rp.get_max_image_dimensions(scored_images)
d_scored_images=[rp.cv_resize_image(x,(d_height,d_width)) for x in scored_images]#,origin='center')
d_scored_images=[rp.labeled_image(im,'%.2f'%score) for im,score in zip(d_scored_images,d_scores)]
rp.display_image_slideshow(d_scored_images[-50:])
rp.display_image_slideshow(d_scored_images[::10])